In [2]:
import org.att.charmander.CharmanderUtils

import org.att.charmander.CharmanderUtils

In [3]:
CharmanderUtils.getMeteredTaskNamesFromRedis

res4: List[String] = List(lookbusy80)

In [6]:
val memoryUsage= CharmanderUtils.getRDDForTask(sc, "lookbusy80", "memory_usage", 20)

memoryUsage: org.apache.spark.rdd.RDD[List[BigDecimal]] = lookbusy80 ParallelCollectionRDD[2] at parallelize at CharmanderUtils.scala:141

In [7]:
memoryUsage.first

res11: List[BigDecimal] = List(1420249001000, 681710001, 84107264)